In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import openpyxl

In [2]:
df = pd.read_excel('/Users/yoeun/Downloads/study/E Commerce Dataset.xlsx', sheet_name=1)

In [3]:
df.head()

,CustomerID,Churn,Tenure,PreferredLoginDevice,CityTier,WarehouseToHome,PreferredPaymentMode,Gender,HourSpendOnApp,NumberOfDeviceRegistered,PreferedOrderCat,SatisfactionScore,MaritalStatus,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,DaySinceLastOrder,CashbackAmount
0,50001,1,4.0,Mobile Phone,3,6.0,Debit Card,Female,3.0,3,Laptop & Accessory,2,Single,9,1,11.0,1.0,1.0,5.0,159.93
1,50002,1,NaN,Phone,1,8.0,UPI,Male,3.0,4,Mobile,3,Single,7,1,15.0,0.0,1.0,0.0,120.90
2,50003,1,NaN,Phone,1,30.0,Debit Card,Male,2.0,4,Mobile,3,Single,6,1,14.0,0.0,1.0,3.0,120.28
3,50004,1,0.0,Phone,3,15.0,Debit Card,Male,2.0,4,Laptop & Accessory,5,Single,8,0,23.0,0.0,1.0,3.0,134.07
4,50005,1,0.0,Phone,1,12.0,CC,Male,NaN,3,Mobile,5,Single,3,0,11.0,1.0,1.0,3.0,129.60


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5630 entries, 0 to 5629
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   CustomerID                   5630 non-null   int64  
 1   Churn                        5630 non-null   int64  
 2   Tenure                       5366 non-null   float64
 3   PreferredLoginDevice         5630 non-null   object 
 4   CityTier                     5630 non-null   int64  
 5   WarehouseToHome              5379 non-null   float64
 6   PreferredPaymentMode         5630 non-null   object 
 7   Gender                       5630 non-null   object 
 8   HourSpendOnApp               5375 non-null   float64
 9   NumberOfDeviceRegistered     5630 non-null   int64  
 10  PreferedOrderCat             5630 non-null   object 
 11  SatisfactionScore            5630 non-null   int64  
 12  MaritalStatus                5630 non-null   object 
 13  NumberOfAddress   

In [5]:
import numpy as np

np.random.seed(42)  # 시드 고정
df['Tenure'].fillna(np.random.randint(61,72), inplace=True)
df['WarehouseToHome'].fillna(0, inplace=True)

df = df[df['HourSpendOnApp'].notna()]
df.drop(columns=['OrderAmountHikeFromlastYear'], inplace=True)
df['CouponUsed'].fillna(0, inplace=True)
df['OrderCount'].fillna(0, inplace=True)
df = df[df['DaySinceLastOrder'].notna()]
df = df[~df['DaySinceLastOrder'].isin([31, 46, 30])] 

/var/folders/b1/g1njf16x0rz622dj3vfvfyy80000gn/T/ipykernel_40844/4143997425.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Tenure'].fillna(np.random.randint(61,72), inplace=True)
/var/folders/b1/g1njf16x0rz622dj3vfvfyy80000gn/T/ipykernel_40844/4143997425.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting 

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5065 entries, 0 to 5629
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CustomerID                5065 non-null   int64  
 1   Churn                     5065 non-null   int64  
 2   Tenure                    5065 non-null   float64
 3   PreferredLoginDevice      5065 non-null   object 
 4   CityTier                  5065 non-null   int64  
 5   WarehouseToHome           5065 non-null   float64
 6   PreferredPaymentMode      5065 non-null   object 
 7   Gender                    5065 non-null   object 
 8   HourSpendOnApp            5065 non-null   float64
 9   NumberOfDeviceRegistered  5065 non-null   int64  
 10  PreferedOrderCat          5065 non-null   object 
 11  SatisfactionScore         5065 non-null   int64  
 12  MaritalStatus             5065 non-null   object 
 13  NumberOfAddress           5065 non-null   int64  
 14  Complain     

In [7]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# 속성이 object인 데이터들의 집합
cat_col = [col for col in df.columns if df[col].dtypes == 'object']

# 속성이 object 아닌 데이터들의 집합
num_col = [col for col in df.columns if df[col].dtypes != 'object']


# 범주형 변수를 숫자로 인코딩 - 추후 predict streamlit 을 위해 하나의 내용으로 변경
label_encoders = {
    'PreferredLoginDevice': LabelEncoder(),
    'PreferredPaymentMode': LabelEncoder(),
    'Gender': LabelEncoder(),
    'PreferedOrderCat': LabelEncoder(),
    'MaritalStatus': LabelEncoder()
}

for col, encoder in label_encoders.items():
    df[col] = encoder.fit_transform(df[col])

# 수치형 변수를 위한 StandardScaler 저장 딕셔너리
scalers = {}

# 수치형 변수 리스트
numeric_columns = ['CityTier', 'WarehouseToHome', 'HourSpendOnApp', 
                   'NumberOfDeviceRegistered', 'SatisfactionScore', 'NumberOfAddress', 'Complain', 
                    'CouponUsed', 'OrderCount','DaySinceLastOrder', 'CashbackAmount']  # 예시 수치형 변수

# 각 수치형 변수에 대해 StandardScaler 생성 및 적용
for col in numeric_columns:
    scaler = StandardScaler()
    # fit_transform을 사용하여 데이터 스케일링
    df[col] = scaler.fit_transform(df[[col]])  # DataFrame 형식으로 전달
    # 각 스케일러를 저장
    scalers[col] = scaler

In [8]:
df.head()

,CustomerID,Churn,Tenure,PreferredLoginDevice,CityTier,WarehouseToHome,PreferredPaymentMode,Gender,HourSpendOnApp,NumberOfDeviceRegistered,PreferedOrderCat,SatisfactionScore,MaritalStatus,NumberOfAddress,Complain,CouponUsed,OrderCount,DaySinceLastOrder,CashbackAmount
0,50001,1,4.0,1,1.479106,-0.983967,4,0,0.113893,-0.678414,2,-0.777578,2,1.840180,1.575910,-0.353570,-0.633891,0.107397,-0.345321
1,50002,1,67.0,2,-0.710481,-0.762603,6,1,0.113893,0.300830,3,-0.052802,2,1.065794,1.575910,-0.901393,-0.633891,-1.289536,-1.133221
2,50003,1,67.0,2,-0.710481,1.672403,4,1,-1.269482,0.300830,3,-0.052802,2,0.678601,1.575910,-0.901393,-0.633891,-0.451376,-1.145737
3,50004,1,0.0,2,1.479106,0.012172,4,1,-1.269482,0.300830,2,1.396749,2,1.452987,-0.634554,-0.901393,-0.633891,-0.451376,-0.867358
5,50006,1,0.0,0,-0.710481,0.786946,4,0,0.113893,1.280074,4,1.396749,2,-0.870172,1.575910,1.289898,1.118270,0.666170,-0.764000


### 모델 구현

In [9]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['CustomerID', 'Churn'])
y = df['Churn']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [10]:
# 결과 확인
print("Train DataFrame:")
print(y_train.value_counts())
print("\nValidation DataFrame:")
print(y_val.value_counts())
print("\nTest DataFrame:")
print(y_test.value_counts())

Train DataFrame:
Churn
0    2709
1     532
Name: count, dtype: int64

Validation DataFrame:
Churn
0    673
1    138
Name: count, dtype: int64

Test DataFrame:
Churn
0    848
1    165
Name: count, dtype: int64


In [139]:
# 1 기본 데이터 기준 학습률

# Logistic
from sklearn.linear_model import LogisticRegression
model_logistic = LogisticRegression()
model_logistic.fit(x_train,y_train)

from sklearn.metrics import classification_report 
print('-----Logistic------')
print(classification_report(y_val, model_logistic.predict(x_val)))

# DecisionTree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

model = DecisionTreeClassifier(random_state=42)

model.fit(x_train, y_train)

y_val_pred = model.predict(x_val)

print('-----DecisionTree------')
print(classification_report(y_val, y_val_pred)) 

# Random forest
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(x_train, y_train)
rfc.score(x_val, y_val)  # accuracy

from sklearn.metrics import classification_report

print('-----RandomForest------')
print(classification_report(y_val, rfc.predict(x_val)))

# XGBoost
import xgboost as xgb

dtrain = xgb.DMatrix(x_train, label=y_train)
dval = xgb.DMatrix(x_val, label=y_val)

params = {
    'objective': 'binary:logistic',  # 이진 분류
    'max_depth': 3,
    'eta': 0.1,
    'eval_metric': 'logloss'
}

model = xgb.train(params, dtrain, num_boost_round=100)

predictions = model.predict(dval)

predictions_binary = [1 if pred > 0.5 else 0 for pred in predictions]

print('-----XGBoost------')
print(classification_report(y_val, predictions_binary))

# CatBoost
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report

model = CatBoostClassifier(verbose=False)
model.fit(x_train, y_train)

y_pred = model.predict(x_val)

print('-----CatBoost------')
print(classification_report(y_val, y_pred))

#MLP
from sklearn.neural_network import MLPClassifier
mlp_model = MLPClassifier(hidden_layer_sizes=(10, 10), max_iter=1000, random_state=42)

mlp_model.fit(x_train, y_train)
y_val_pred = mlp_model.predict(x_val)

print('-----MLP------')
print(classification_report(y_val, y_val_pred))

#GradientBoosting
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

model = GradientBoostingClassifier(random_state=42)

model.fit(x_train, y_train)

y_val_pred = model.predict(x_val)

print('-----GradientBoosting------')
print(classification_report(y_val, y_val_pred))

#AdaBoost
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report

model = AdaBoostClassifier(random_state=42)

model.fit(x_train, y_train)

y_val_pred = model.predict(x_val)

print('-----AdaBoost------')
print(classification_report(y_val, y_val_pred))

-----Logistic------
              precision    recall  f1-score   support

           0       0.87      0.98      0.93       673
           1       0.80      0.31      0.45       138

    accuracy                           0.87       811
   macro avg       0.84      0.65      0.69       811
weighted avg       0.86      0.87      0.84       811

-----DecisionTree------
              precision    recall  f1-score   support

           0       0.96      0.96      0.96       673
           1       0.79      0.78      0.79       138

    accuracy                           0.93       811
   macro avg       0.87      0.87      0.87       811
weighted avg       0.93      0.93      0.93       811

-----RandomForest------
              precision    recall  f1-score   support

           0       0.96      0.99      0.97       673
           1       0.95      0.78      0.86       138

    accuracy                           0.96       811
   macro avg       0.95      0.89      0.92       811
weight

/opt/homebrew/Caskroom/miniforge/base/envs/dlpjt/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [12]:
# 1 클래스의 정확도 부족으로 오버샘플링
from imblearn.over_sampling import RandomOverSampler

rds = RandomOverSampler(random_state=42)
x_train_resample, y_train_resample = rds.fit_resample(x_train,y_train)
print(f'shape : {x_train.shape}, {x_train_resample.shape}')


# Logistic
from sklearn.linear_model import LogisticRegression
model_logistic = LogisticRegression()
model_logistic.fit(x_train_resample,y_train_resample)

from sklearn.metrics import classification_report 
print('-----Logistic------')
print(classification_report(y_val, model_logistic.predict(x_val)))

# DecisionTree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

model = DecisionTreeClassifier(random_state=42)

model.fit(x_train_resample, y_train_resample)

y_val_pred = model.predict(x_val)

print('-----DecisionTree------')
print(classification_report(y_val, y_val_pred)) 

# Random forest
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(x_train_resample, y_train_resample)
rfc.score(x_val, y_val)  # accuracy

from sklearn.metrics import classification_report

print('-----RandomForest------')
print(classification_report(y_val, rfc.predict(x_val)))

# XGBoost
import xgboost as xgb

dtrain = xgb.DMatrix(x_train_resample, label=y_train_resample)
dval = xgb.DMatrix(x_val, label=y_val)

params = {
    'objective': 'binary:logistic',  # 이진 분류
    'max_depth': 3,
    'eta': 0.1,
    'eval_metric': 'logloss'
}

model = xgb.train(params, dtrain, num_boost_round=100)

predictions = model.predict(dval)

predictions_binary = [1 if pred > 0.5 else 0 for pred in predictions]

print('-----XGBoost------')
print(classification_report(y_val, predictions_binary))

# CatBoost
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report

model = CatBoostClassifier(verbose=False)
model.fit(x_train_resample, y_train_resample)

y_pred = model.predict(x_val)

print('-----CatBoost------')
print(classification_report(y_val, y_pred))

#MLP
from sklearn.neural_network import MLPClassifier
mlp_model = MLPClassifier(hidden_layer_sizes=(10, 10), max_iter=1000, random_state=42)

mlp_model.fit(x_train_resample, y_train_resample)
y_val_pred = mlp_model.predict(x_val)

print('-----MLP------')
print(classification_report(y_val, y_val_pred))

#GradientBoosting
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

model = GradientBoostingClassifier(random_state=42)

model.fit(x_train_resample, y_train_resample)

y_val_pred = model.predict(x_val)

print('-----GradientBoosting------')
print(classification_report(y_val, y_val_pred))

#AdaBoost
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report

model = AdaBoostClassifier(random_state=42)

model.fit(x_train_resample, y_train_resample)

y_val_pred = model.predict(x_val)

print('-----AdaBoost------')
print(classification_report(y_val, y_val_pred))

shape : (3241, 17), (5418, 17)
-----Logistic------
              precision    recall  f1-score   support

           0       0.94      0.71      0.81       673
           1       0.35      0.76      0.48       138

    accuracy                           0.72       811
   macro avg       0.64      0.74      0.65       811
weighted avg       0.84      0.72      0.75       811

-----DecisionTree------
              precision    recall  f1-score   support

           0       0.96      0.97      0.97       673
           1       0.86      0.79      0.82       138

    accuracy                           0.94       811
   macro avg       0.91      0.88      0.89       811
weighted avg       0.94      0.94      0.94       811

-----RandomForest------
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       673
           1       0.93      0.83      0.87       138

    accuracy                           0.96       811
   macro avg       0.95      

/opt/homebrew/Caskroom/miniforge/base/envs/dlpjt/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

# MLP 모델 설정
model = MLPClassifier(max_iter=5000, random_state=42, tol=1e-5)  # max_iter와 tol 조정

# 하이퍼파라미터 그리드 설정
param_grid = {
    'hidden_layer_sizes': [(10,)], #(20,), (10, 10)],  # 작은 네트워크 구조만 남김
    'activation': ['relu', 'tanh'],  # 자주 사용되는 두 가지 활성화 함수
    'solver': ['adam', 'sgd'],  # 두 가지 주요 최적화 방법
    'alpha': [0.0001, 0.001, 0.01]  # 적절한 정규화 값으로 범위 축소
}

# 그리드 서치 설정
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1)

# 그리드 서치 실행
grid_search.fit(x_train_resample, y_train_resample)

# 최적의 파라미터 및 성능 출력
print("최적의 파라미터:", grid_search.best_params_)
print("최고 정확도:", grid_search.best_score_)

# 테스트 데이터에 대한 예측
y_pred = grid_search.predict(x_val)

# 성능 평가
print("테스트 데이터 성능 보고서:")
print(classification_report(y_val, y_pred))

In [151]:
xgb_params = {
    'max_depth': [3, 5, 7, 9],  # 트리의 최대 깊이
    'learning_rate': [0.1],  # 학습률
    'n_estimators': [100, 200, 300],  # 부스팅 라운드 수
    'subsample': [0.5, 0.7],  # 각 트리의 훈련에 사용되는 샘플 비율
    'colsample_bytree': [0.5, 0.7, 1.0],  # 각 트리의 훈련에 사용되는 피처 비율
    'gamma': [0, 0.1],  # 노드 분할에 대한 최소 손실 감소
    'reg_alpha': [0],  # L1 정규화
    'reg_lambda': [0.1]  # L2 정규화
}

from sklearn.model_selection import GridSearchCV
import xgboost as xgb

xgb_model = xgb.XGBClassifier(eval_metric='logloss')

xgb_grid = GridSearchCV(estimator=xgb_model, param_grid=xgb_params, scoring='accuracy', cv=5)
xgb_grid.fit(x_train_resample, y_train_resample)

print(f'Best Parameters for XGBoost: {xgb_grid.best_params_}')
print(f'Best Score: {xgb_grid.best_score_}')

# 최적 모델로 예측 수행
best_model = xgb_grid.best_estimator_
predictions = best_model.predict(x_val)

# 이진 예측 결과 출력
print(classification_report(y_val, predictions))


Best Parameters for XGBoost: {'colsample_bytree': 0.5, 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 9, 'n_estimators': 200, 'reg_alpha': 0, 'reg_lambda': 0.1, 'subsample': 0.5}
Best Score: 0.988004313561141
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       673
           1       0.93      0.87      0.90       138

    accuracy                           0.97       811
   macro avg       0.95      0.93      0.94       811
weighted avg       0.97      0.97      0.97       811



In [160]:
# CatBoost

cat_params = {
    'iterations': [100, 200, 500],  # 학습 반복 수
    'depth': [3, 5, 7, 10],  # 트리의 깊이
    'learning_rate': [0.1, 0.3],  # 학습률
    'l2_leaf_reg': [1, 3, 5],  # L2 정규화
    'border_count': [32, 64],  # 경계 수
    'bagging_temperature': [0, 0.5, 1],  # 배깅 온도
    'early_stopping_rounds': [10]  # 조기 중단 라운드 수
}

from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier

cat_model = CatBoostClassifier(verbose=False)

cat_grid = GridSearchCV(estimator=cat_model, param_grid=cat_params, scoring='accuracy', cv=5)
cat_grid.fit(x_train_resample, y_train_resample)

print(f'Best Parameters for CatBoost: {cat_grid.best_params_}')
print(f'Best Score: {cat_grid.best_score_}')

# 최적 모델로 예측 수행
best_model = cat_grid.best_estimator_
predictions = best_model.predict(x_val)

# 이진 예측 결과 출력
print(classification_report(y_val, predictions))

Best Parameters for CatBoost: {'bagging_temperature': 0, 'border_count': 32, 'depth': 10, 'early_stopping_rounds': 10, 'iterations': 500, 'l2_leaf_reg': 1, 'learning_rate': 0.1}
Best Score: 0.9904031782700098
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       673
           1       0.92      0.86      0.89       138

    accuracy                           0.96       811
   macro avg       0.95      0.92      0.93       811
weighted avg       0.96      0.96      0.96       811



In [ ]:
# randomforest

# 하이퍼파라미터 그리드 정의
param_grid = {
    'n_estimators': [50, 100, 200],            # 결정 트리의 수
    'max_depth': [None, 10, 20, 30],           # 트리의 최대 깊이
    'min_samples_split': [2, 5, 10],           # 노드를 분할하는 데 필요한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4],             # 리프 노드에 필요한 최소 샘플 수
    'max_features': ['auto', 'sqrt', 'log2']  # 최적의 분할을 찾기 위해 고려할 특성 수
}

# GridSearchCV 객체 생성
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                           param_grid=param_grid,
                           scoring='accuracy',  # 평가 지표
                           cv=5,                # 5-폴드 교차 검증
                           verbose=2,          # 출력 레벨
                           n_jobs=-1)          # 모든 CPU 사용

# GridSearchCV 훈련
grid_search.fit(x_train_resample, y_train_resample)

# 최적의 하이퍼파라미터 및 교차 검증 점수 출력
print("최적의 하이퍼파라미터:", grid_search.best_params_)
print("최고의 교차 검증 점수:", grid_search.best_score_)

# 최적 모델로 검증 데이터 평가
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_val)

# 분류 보고서 출력
print("검증 데이터에 대한 분류 보고서:")
print(classification_report(y_val, y_pred))

In [ ]:
# 추후 라벨 인코딩 및 scaler inverse

# 예시: 복원
# 복원할 데이터프레임 생성 (예: 인코딩된 데이터)
encoded_values = df[['PreferredLoginDevice', 'PreferredPaymentMode']].head(5)

# 복원
for col, encoder in label_encoders.items():
    if col in encoded_values.columns:
        encoded_values[col] = encoder.inverse_transform(encoded_values[col])

print(encoded_values)
# 예시: 복원
# 복원할 데이터프레임 생성 (예: 스케일된 데이터)
scaled_values = df[numeric_columns].head(5)

# 복원
for col in numeric_columns:
    if col in scalers:
        scaled_values[col] = scalers[col].inverse_transform(scaled_values[[col]])  # DataFrame 형식으로 전달

print(scaled_values) 